# Edges with Polymake.jl

We compute the edges of the cube using Polymake.jl in Julia

In [1]:
using Polymake

polymake version 4.4
Copyright (c) 1997-2021
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



We start by making a cube:

In [2]:
c = Polymake.polytope.cube(3)

type: Polytope<Rational>
description: cube of dimension 3

AFFINE_HULL
	

BOUNDED
	true

CONE_AMBIENT_DIM
	4

CONE_DIM
	4

FACETS
  1   1   0   0
  1  -1   0   0
  1   0   1   0
  1   0  -1   0
  1   0   0   1
  1   0   0  -1

VERTICES_IN_FACETS
	{0 2 4 6}
	{1 3 5 7}
	{0 1 4 5}
	{2 3 6 7}
	{0 1 2 3}
	{4 5 6 7}

Edges of the cube are its one-dimensional faces. To access these we may use the `faces_of_dim` method:

In [5]:
Polymake.polytope.faces_of_dim(c, 1)

pm::Array<pm::Set<long, pm::operations::cmp>>
{0 4}
{2 6}
{0 2}
{4 6}
{1 5}
{3 7}
{1 3}
{5 7}
{0 1}
{4 5}
{2 3}
{6 7}


The encoding again means that `{0 4}` corresponds to an edge between the 0th and the 4th vertex of the cube.

However, Julia uses `1` based encodings, so we can use a convenience function of Polymake.jl to fix this issue:

In [40]:
Polymake.to_one_based_indexing(Polymake.polytope.faces_of_dim(c,1))

12-element Vector{Polymake.SetAllocated{Int64}}:
 pm::Set<long, pm::operations::cmp>
{1 5}
 pm::Set<long, pm::operations::cmp>
{3 7}
 pm::Set<long, pm::operations::cmp>
{1 3}
 pm::Set<long, pm::operations::cmp>
{5 7}
 pm::Set<long, pm::operations::cmp>
{2 6}
 pm::Set<long, pm::operations::cmp>
{4 8}
 pm::Set<long, pm::operations::cmp>
{2 4}
 pm::Set<long, pm::operations::cmp>
{6 8}
 pm::Set<long, pm::operations::cmp>
{1 2}
 pm::Set<long, pm::operations::cmp>
{5 6}
 pm::Set<long, pm::operations::cmp>
{3 4}
 pm::Set<long, pm::operations::cmp>
{7 8}

Now the first entry becomes `{1 5}` and indicates an edge between the first and the fifth vertex.

Accessing and conversion can be done in the following way:

In [41]:
A = Polymake.to_one_based_indexing(Polymake.polytope.faces_of_dim(c, 1));
println(Set{Int}(A[1]))
println(Vector{Int}(A[1]))

Set([5, 1])
[1, 5]


The edge directions would have to be computed separately, e.g. in the following way:

(I cheated a little, by choosing `Int` for the type of the matrix entries, `Rational` would be more honest, but less aesthetic)

In [42]:
V = Matrix{Int}(c.VERTICES)
for edge in A
    v = Vector{Int}(edge)
    dir = V[v[1], :] - V[v[2], :]
    println("$(v) has dir $(dir)")
end

[1, 5] has dir [0, 0, 0, -2]
[3, 7] has dir [0, 0, 0, -2]
[1, 3] has dir [0, 0, -2, 0]
[5, 7] has dir [0, 0, -2, 0]
[2, 6] has dir [0, 0, 0, -2]
[4, 8] has dir [0, 0, 0, -2]
[2, 4] has dir [0, 0, -2, 0]
[6, 8] has dir [0, 0, -2, 0]
[1, 2] has dir [0, -2, 0, 0]
[5, 6] has dir [0, -2, 0, 0]
[3, 4] has dir [0, -2, 0, 0]
[7, 8] has dir [0, -2, 0, 0]


One can see that the edge directions are the correct ones, i.e. only one non-zero entry.